# 저장된 모델 활용 
- 모델 파일 종류 
    - 가중치 및 절편 저장 파일 => 동일한 구조 모델 인스턴스 생성 후 사용 ㅇ 
    - 모델 전체 저장 파일 => 바로 로딩 후 사용 ㅇ 


## [1] 모듈 로딩 
---

In [10]:
# 모듈 로딩 

import torch 
import torch.nn as nn
import torch.nn.functional as F
from torchinfo import summary

In [16]:
# 모델 파일 관련 

# models 폴더 아래 프로젝트 폴더 아래 모델 파일 저장 
import os 

# 저장 경로 
SAVE_PATH = '../MODELS/BCF/'  

# 저장 파일명 
SAVE_FILE = SAVE_PATH + 'model_train_wbs.pth'

# 모델 구조 및 파라미터 모두 저장 파일명 
SAVE_MODEL = SAVE_PATH + 'model_all.pth'

## [2] 모델 로딩 - 모델 전체 파일 사용 

In [12]:
class iris_bs_model(nn.Module):
    
    # 모델 구조 구성 및 인스턴스 생성 및 메서드 
    def __init__(self):
        super().__init__()

        # 모델 구조 구성 
        self.in_layer = nn.Linear(4,10)
        self.hidden_layer = nn.Linear(10,5)
        self.out_layer = nn.Linear(5,1)

    # 순방향 학습 진행 메서드 
    def forward(self, input_data):

        # 입력층 
        y = self.in_layer(input_data) # f11w11 + f12w12 + f13w13 + b, ......., f101w101 + f102w102 + f103w103 + b
        y = F.relu(y)             # relu => y 값의 범위 : 0 <= y 

        # 은닉층 : 10개의 숫자 값(>=0)
        y = self.hidden_layer(y)  # f21w11 + f22w12 .... + f210w210 + b, ......., f230w201 + f230w202 ..... f230w210 + b
        # 데이터 1개 기준
        y = F.relu(y) 

        # 출력층 : 5개의 숫자 값(>=0)
        # self.out_layer(y)         # f31w31 + ...... f330w330 + b
        # 회귀라서 활성함수 사용 x -> 바로 return 

        return F.sigmoid(self.out_layer(y))


In [13]:
iris_model = torch.load(SAVE_MODEL, weights_only=False )


In [14]:
summary(iris_model)


Layer (type:depth-idx)                   Param #
iris_bs_model                            --
├─Linear: 1-1                            50
├─Linear: 1-2                            55
├─Linear: 1-3                            6
Total params: 111
Trainable params: 111
Non-trainable params: 0

## [3] 예측 
---

In [17]:
data = [float(x) for x in input('SL, SW, PL, PW:' ).split(',')]

In [18]:
data_ts = torch.FloatTensor(data).reshape(1,-1)
data_ts.shape, data_ts

(torch.Size([1, 4]), tensor([[0.1200, 0.4400, 0.3200, 0.6600]]))

In [19]:
# 새로운 데이터에 대한 예측 즉, predict  

iris_model.eval()
with torch.no_grad():

    # 추론/평가 
    pre_val = iris_model(data_ts)


In [20]:
pre_val

tensor([[0.5021]])

In [21]:
'Other' if (pre_val > 0.5).item() else 'Setosa'
# 0.5 보다 크면 음성 값 

'Other'